In [116]:
import pyspark
from pyspark import SparkConf, SparkContext
import pandas as pd

ModuleNotFoundError: No module named 'torch'

In [2]:
conf = (SparkConf().setMaster("yarn")
        .setAppName("s4528239-final_project"))

In [3]:
sc = SparkContext(conf = conf)

In [4]:
sc.sparkUser()

's4528239'

In [5]:
! hdfs dfs -ls /data/ProjectDatasetTwitter

Found 184 items
-rwxr-xr-x   3 hdfs hdfs 2305213666 2018-02-12 07:19 /data/ProjectDatasetTwitter/statuses.log.2014-07-01.gz
-rwxr-xr-x   3 hdfs hdfs 2179827156 2018-02-12 07:19 /data/ProjectDatasetTwitter/statuses.log.2014-07-02.gz
-rwxr-xr-x   3 hdfs hdfs 2175316090 2018-02-12 07:20 /data/ProjectDatasetTwitter/statuses.log.2014-07-03.gz
-rwxr-xr-x   3 hdfs hdfs 2245412054 2018-02-12 07:20 /data/ProjectDatasetTwitter/statuses.log.2014-07-04.gz
-rwxr-xr-x   3 hdfs hdfs 2172924772 2018-02-12 07:20 /data/ProjectDatasetTwitter/statuses.log.2014-07-05.gz
-rwxr-xr-x   3 hdfs hdfs 2192938521 2018-02-12 07:20 /data/ProjectDatasetTwitter/statuses.log.2014-07-06.gz
-rwxr-xr-x   3 hdfs hdfs 2240548209 2018-02-12 07:20 /data/ProjectDatasetTwitter/statuses.log.2014-07-07.gz
-rwxr-xr-x   3 hdfs hdfs 2530331847 2018-02-12 07:21 /data/ProjectDatasetTwitter/statuses.log.2014-07-08.gz
-rwxr-xr-x   3 hdfs hdfs 2352936093 2018-02-12 07:21 /data/ProjectDatasetTwitter/statuses.log.2014-07-09.gz
-rwxr-xr-x  

In [6]:
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import mean, min, max, lit, col, first, last, sum, count, countDistinct, desc

In [7]:
spark = SparkSession \
    .builder \
    .appName("project") \
    .getOrCreate()

In [19]:
df = spark.read.format("json").load("/data/ProjectDatasetTwitter/statuses.log.2014-07-01.gz")

In [33]:
df.columns

['contributors',
 'coordinates',
 'created_at',
 'delete',
 'entities',
 'favorite_count',
 'favorited',
 'filter_level',
 'geo',
 'id',
 'id_str',
 'in_reply_to_screen_name',
 'in_reply_to_status_id',
 'in_reply_to_status_id_str',
 'in_reply_to_user_id',
 'in_reply_to_user_id_str',
 'lang',
 'place',
 'possibly_sensitive',
 'retweet_count',
 'retweeted',
 'retweeted_status',
 'scopes',
 'source',
 'status_withheld',
 'text',
 'truncated',
 'user',
 'withheld_in_countries']

In [42]:
df.printSchema()

root
 |-- contributors: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- delete: struct (nullable = true)
 |    |-- status: struct (nullable = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- id_str: string (nullable = true)
 |    |    |-- user_id: long (nullable = true)
 |    |    |-- user_id_str: string (nullable = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- display_url: string (nullable = true)


In [43]:
df.createOrReplaceTempView("table1")

In [280]:
spark.sql('select text from table1 where geo is not null').show(20,False)

+-------------------------------------------------------------------------------------------------+
|text                                                                                             |
+-------------------------------------------------------------------------------------------------+
|@MarkArmour7 wait to a get ma hands on him                                                       |
|@NadeemTellawi ba3d galbi bu nodom&lt;3                                                          |
|That was a waste of time                                                                         |
|Melhor parte de estar rouca é não ter que responder as pessoas para poupar a voz                 |
|@NikeFuel toutafé ! Go 🇫🇷                                                                      |
|Alhamdulillah lebaran 28 hari lagi .                                                             |
|@Kalbi_temiz: Sevdiklerimizi sana emânet ediyoruz Allah'ım, Zirâ sen; emanetleri kaybetmeyensin..|
|A

In [9]:
# query all tweets about China, and Chinese
df_all = spark.read.format("json").load("/data/ProjectDatasetTwitter/statuses.log.2014-07-01.gz")
df_all.createOrReplaceTempView("table_first")
df_all_tweets = spark.sql('select text from table_first where \
text like "% china %" or text like "% CHINA %" or text like "% China %" or text like "% Chinese %" or text like "% chinese %" \
or array_contains(entities.hashtags.text,"China") or array_contains(entities.hashtags.text,"CHINA") \
or array_contains(entities.hashtags.text,"Chinese") or  array_contains(entities.hashtags.text,"chinese") or \
array_contains(entities.hashtags.text,"china")')


date_period = pd.date_range('7/1/2014', '7/7/2014')
for i in date_period[1:]:
    the_date = str(i)[0:10]
    df_new = spark.read.format("json").load("/data/ProjectDatasetTwitter/statuses.log." + the_date +".gz")
    new_table_name = "table_" + the_date[0:4] + the_date[5:7] + the_date[8:10]
    df_new.createOrReplaceTempView(new_table_name)
    df_new_tweets = spark.sql('select text from ' + new_table_name + ' where \
text like "% china %" or text like "% CHINA %" or text like "% China %" or text like "% Chinese %" or text like "% chinese %" \
or array_contains(entities.hashtags.text,"China") or array_contains(entities.hashtags.text,"CHINA") \
or array_contains(entities.hashtags.text,"Chinese") or  array_contains(entities.hashtags.text,"chinese") or \
array_contains(entities.hashtags.text,"china")')
    df_all_tweets = df_all_tweets.union(df_new_tweets)
    print(the_date)

2014-07-02
2014-07-03
2014-07-04
2014-07-05
2014-07-06
2014-07-07


In [10]:
df_all_tweets_pandas = df_all_tweets.toPandas()

In [14]:
df_all_tweets_pandas

,text
0,Come join Diane Hall of Diane's Kitchen on Jul...
1,A Chinese Man Smashed His Brand New Model S To...
2,。南京大虐殺；(http://t.co/ZxgKzTbuFR ) Chinese gover...
3,RT @garrymula: مشاهدة الأفلام مجانا الافراج عن...
4,RT @garrymula: مشاهدة الأفلام مجانا الافراج عن...
...,...
10679,RT @CancilleriaPeru: Canciller de #China felic...
10680,RT @WangFei7u7: Bang me cuida a la china ¿? ( ...
10681,El Puerto #Nayarit será presentado formalmente...
10682,RT @FootballFunnys: There are two things visib...


In [16]:
(10684 / 35000000)*100

0.030525714285714284

In [22]:
# select tweets about America or united states
df_all = spark.read.format("json").load("/data/ProjectDatasetTwitter/statuses.log.2014-07-01.gz")
df_all.createOrReplaceTempView("table_first")
df_all_tweets = spark.sql('select text from ' + new_table_name + ' where \
text like "% Japanese %" or text like "% JAPAN %" or text like "% Japan %" or text like "% Japanese %" or text like "% Japaneses %" \
or array_contains(entities.hashtags.text,"JAPAN") or array_contains(entities.hashtags.text,"Japan") \
or array_contains(entities.hashtags.text,"Japanese") or  array_contains(entities.hashtags.text,"Japaneses") or \
array_contains(entities.hashtags.text,"japan")')

date_period = pd.date_range('7/1/2014', '7/7/2014')
for i in date_period[1:]:
    the_date = str(i)[0:10]
    df_new = spark.read.format("json").load("/data/ProjectDatasetTwitter/statuses.log." + the_date +".gz")
    new_table_name = "table_" + the_date[0:4] + the_date[5:7] + the_date[8:10]
    df_new.createOrReplaceTempView(new_table_name)
    df_new_tweets = spark.sql('select text from ' + new_table_name + ' where \
text like "% Japanese %" or text like "% JAPAN %" or text like "% Japan %" or text like "% Japanese %" or text like "% Japaneses %" \
or array_contains(entities.hashtags.text,"JAPAN") or array_contains(entities.hashtags.text,"Japan") \
or array_contains(entities.hashtags.text,"Japanese") or  array_contains(entities.hashtags.text,"Japaneses") or \
array_contains(entities.hashtags.text,"japan")')
    df_all_tweets = df_all_tweets.union(df_new_tweets)
    print(the_date)
df_all_tweets_about_Japan_pandas = df_all_tweets.toPandas()

2014-07-02
2014-07-03
2014-07-04
2014-07-05
2014-07-06
2014-07-07


In [23]:
df_all_tweets_about_Japan_pandas

,text
0,My Japan Name isNagami ShiroganeMy English Nam...
1,Bearded Pussy Of Japanese Cutie SERINA HAYAKAW...
2,Japanese silk Obi for Kimono http://t.co/MvBG0...
3,@Godheberuling Japanese has cooler looking cha...
4,【拡散希望】ROCK IN JAPAN FES 2014 8/9.10の後2日券2枚お譲り...
...,...
9458,"""@ABC: Japanese island of Okinawa on alert as ..."
9459,Arriving in Tokyo http://t.co/KQQolyeXhI #http...
9460,Mesmerizing Japanese Babe Pokes Her Bearded Va...
9461,RT @kpop_lov: 【WINNER 1st JAPAN TOUR 2014】\n\n...


In [24]:
(9463 / 35000000) * 100

0.027037142857142857

In [36]:
df_all_tweets_pandas

,text
0,Come join Diane Hall of Diane's Kitchen on Jul...
1,A Chinese Man Smashed His Brand New Model S To...
2,。南京大虐殺；(http://t.co/ZxgKzTbuFR ) Chinese gover...
3,RT @garrymula: مشاهدة الأفلام مجانا الافراج عن...
4,RT @garrymula: مشاهدة الأفلام مجانا الافراج عن...
...,...
10679,RT @CancilleriaPeru: Canciller de #China felic...
10680,RT @WangFei7u7: Bang me cuida a la china ¿? ( ...
10681,El Puerto #Nayarit será presentado formalmente...
10682,RT @FootballFunnys: There are two things visib...


In [39]:
list_all_tweets_about_China = df_all_tweets_pandas['text']

In [44]:
the_first_str = list_all_tweets_about_China[0]
for i in list_all_tweets_about_China[1:]:
    the_first_str = the_first_str + ' '+ i
union_str_about_China = the_first_str

In [49]:
# split the text and do the word counting
test_str = str(union_str_about_China)
import re
array=re.split('[ ,.\n]',test_str)

dic={}
for i in array:
    if i not in dic:
        dic[i] = 1
    else:
        dic[i] += 1
list1= sorted(dic.items(),key=lambda x:x[1],reverse = True)
print('\n\nword count：',list1)



word count： [('', 13902), ('http://t', 6987), ('RT', 4816), ('China', 4223), ('Chinese', 3295), ('in', 2175), ('the', 2062), ('to', 1895), ('a', 1665), ('#China', 1303), ('of', 1292), ('and', 1265), ('china', 1012), ('for', 930), ('-', 885), ('I', 884), ('is', 848), ('de', 741), ('chinese', 698), ('by', 644), ('fast', 633), ('you', 611), ('@Gabriele_Corno:', 605), ('Free', 597), ('on', 595), ('#VPN:', 590), ('مشاهدة', 589), ('الأفلام', 589), ('مجانا', 589), ('الافراج', 589), ('عن', 589), ('المواقع', 589), ('food', 559), ('la', 477), ('from', 476), ('@', 476), ('with', 461), ('en', 455), ('The', 434), ('In', 429), ('at', 421), ('me', 397), ('#Chinese', 389), ('that', 378), ('#Hongkong', 368), ('are', 350), ('&amp;', 337), ('my', 326), ('y', 317), ('#GFW', 285), ('#FuckGFW', 284), ('que', 276), ('be', 271), ('it', 264), ('Beijing', 264), ('as', 235), ('this', 235), ('via', 229), ('A', 224), ('but', 224), ('i', 223), ('some', 217), ('#china', 216), ('like', 212), ('Moon', 211), ('con', 

In [52]:
# analyzing tweets about "china" and "fast"
tweets_about_china_and_fast = []
vpn_count = 0
for i in list_all_tweets_about_China:
    if "fast" in i:
        tweets_about_china_and_fast.append(i)
    if "VPN" in i:
        vpn_count = vpn_count + 1

In [55]:
len(tweets_about_china_and_fast)

733

In [56]:
tweets_about_china_and_fast

['RT @garrymula: مشاهدة الأفلام مجانا الافراج عن المواقع \n\nFree fast #VPN: http://t.co/dxVpZ6KofU\n#Chinese #Hongkong',
 'RT @garrymula: مشاهدة الأفلام مجانا الافراج عن المواقع \n\nFree fast #VPN: http://t.co/dxVpZ6KofU\n#Chinese #Hongkong',
 'RT @garrymula: مشاهدة الأفلام مجانا الافراج عن المواقع \n\nFree fast #VPN: http://t.co/dxVpZ6KofU\n#Chinese #Hongkong',
 'RT @garrymula: مشاهدة الأفلام مجانا الافراج عن المواقع \n\nFree fast #VPN: http://t.co/dxVpZ6KofU\n#Chinese #Hongkong',
 'RT @garrymula: مشاهدة الأفلام مجانا الافراج عن المواقع \n\nFree fast #VPN: http://t.co/dxVpZ6KofU\n#Chinese #Hongkong',
 'RT @fgi35891041: مشاهدة الأفلام مجانا الافراج عن المواقع \n\nFree fast #VPN: http://t.co/d40DQ5AoDE\n#Chinese #Hongkong',
 'RT @fgi35891041: مشاهدة الأفلام مجانا الافراج عن المواقع \n\nFree fast #VPN: http://t.co/d40DQ5AoDE\n#Chinese #Hongkong',
 'RT @fgi35891041: مشاهدة الأفلام مجانا الافراج عن المواقع \n\nFree fast #VPN: http://t.co/d40DQ5AoDE\n#Chinese #Hongkong',
 'RT @fgi35891041:

In [53]:
vpn_count

590

In [57]:
# analyzing tweets about a name
tweets_about_a_name = []
vpn_count = 0
for i in list_all_tweets_about_China:
    if "Gabriele_Corno" in i:
        tweets_about_a_name.append(i)

In [59]:
tweets_about_a_name_set = set(tweets_about_a_name)

In [60]:
tweets_about_a_name_set

{'RT @BonVoyageurs: SAVVY TIP: Check with http://t.co/kQDPpn5BcT for tickets to #China or #Asia! I always do! Pic by @Gabriele_Corno http://t…',
 'RT @Gabriele_Corno: Fisherman In Yangshuo by Peter Stewart #China http://t.co/CrlXNtInrP',
 'RT @Gabriele_Corno: Moon bridge @ Beijing by Claro Cortes #China http://t.co/BbhxZVpzMB',
 'RT @Gabriele_Corno: Tea Fields @ Zhejiang, China http://t.co/KpjAKyBRav'}

In [61]:
len(tweets_about_a_name)

606

In [69]:
# analyzing tweets about food
all_tweets_about_food = []
for i in list_all_tweets_about_China:
    if "food" in i:
        all_tweets_about_food.append(i)
len(all_tweets_about_food)

608

In [70]:
all_tweets_about_food

['I want some chinese food..',
 'I now have the sudden urge to eat Chinese food',
 'I want some Chinese food😌',
 'Finally got my Chinese food 👌',
 "My co worker ordered Chinese food &amp; didn't tell me &amp; I'm just like #starvinguntil9 http://t.co/JVxMNsf3vs",
 '@Ohso_RackedUp chinese food',
 'Ate too much Chinese food 😩 feel so sick',
 'Coming home from work to some Chinese food😻',
 '@ZoamSc2 make sandwich w/ Chinese food in it, duh',
 'Some Chinese food now',
 'Can my mom hurry up &amp; bring me my chinese food.',
 '@BrokenStonerrr every thing but I really love Chinese food',
 'But he said he was going to wear some dragon shorts with me and take me to eat chinese food lol.',
 '@monomakia ok let me get chinese food and you get to decide which movie too',
 'Dude I freaking maxed that Chinese food.',
 'we got chinese food thanks',
 'I want chinese food :(',
 'RT @Oh_Its_Maddieee: Going get chinese food !',
 'I love Chinese food.',
 'I really want Chinese food or cheesecake',
 'RT @fo

In [72]:
# count "chinese food" tweets
chinese_food_count = 0
for j in all_tweets_about_food:
    if "Chinese food" in j:
        chinese_food_count = chinese_food_count + 1
chinese_food_count

392

In [73]:
len(all_tweets_about_food)

608

In [74]:
tweets_about_chinese_food = []
for j in all_tweets_about_food:
    if "Chinese food" in j:
        tweets_about_chinese_food.append(j)
len(tweets_about_chinese_food)

392

In [94]:
tweets_about_chinese_food

['I now have the sudden urge to eat Chinese food',
 'I want some Chinese food😌',
 'Finally got my Chinese food 👌',
 "My co worker ordered Chinese food &amp; didn't tell me &amp; I'm just like #starvinguntil9 http://t.co/JVxMNsf3vs",
 'Ate too much Chinese food 😩 feel so sick',
 'Coming home from work to some Chinese food😻',
 '@ZoamSc2 make sandwich w/ Chinese food in it, duh',
 'Some Chinese food now',
 '@BrokenStonerrr every thing but I really love Chinese food',
 'Dude I freaking maxed that Chinese food.',
 'I love Chinese food.',
 'I really want Chinese food or cheesecake',
 'I want some Chinese food',
 'I love Chinese food',
 '“@JonahhMahone: Eatin Chinese food with the brown sauce” soy sauce?',
 '@TheCBurns sketchy Chinese food',
 'me ordering Chinese food http://t.co/tTCucpzJoU',
 'I feel bad if your idea of good Chinese food is Panda Express',
 "I'm craving Chinese food 😁😋",
 'Fucking Chinese food❤️',
 "@__2fab Oh ighh lol I'm maccin juss put to rest my Chinese food \ue11c lol t

## Sentiment Analysis（all the following code are runned on Google Colab）
## Colab address:
https://colab.research.google.com/drive/1Kb5Ma7cNPBk52Buaxh4KWj6Q1MtzButH?usp=sharing

In [ ]:
import torch
from transformers import pipeline

# filter emoji in the tweets
chinese_food_tweets_new = []
for i in tweets_about_chinese_food:
    tweets_after_filter_emoji = str(bytes(i, encoding='utf-8').decode('utf-8').encode('gbk', 'ignore').decode('gbk'))
    chinese_food_tweets_new.append(tweets_after_filter_emoji)

nlp = pipeline("sentiment-analysis")

count_positive = 0
count_negative = 0
count_neutral = 0
for i in chinese_food_tweets_new:
      output = nlp(i)[0]
    if output['score'] >= 0.4 and output['score'] <= 0.6:
        count_neutral = count_neutral + 1
        continue
    if output['label'] == "POSITIVE":
        count_positive = count_positive + 1
        continue
    if output['label'] == "NEGATIVE":
        count_negative = count_negative + 1
        continue